---
# Vertex AI
---

## Setup
---
Define constants

https://codelabs.developers.google.com/vertex-training-autopkg#5

+ autopkg-codelab
    + requirements.txt
    + trainer/
        + task.py

In [1]:
MODEL_DIR='autopkg-codelab'

In [2]:
%%bash
mkdir autopkg-codelab
mkdir autopkg-codelab/trainer
cd autopkg-codelab

---
## 1. Write trainer/task.py
---

In [3]:
%%writefile {MODEL_DIR}/trainer/task.py
import argparse
import numpy as np 
import pandas 
import csv
import warnings
from google.cloud import storage
from io import BytesIO
import os
import string
import time
import pickle
import seaborn as sns
from pathlib import Path
import transformers
import torch
import random
import pyarrow as pa
import datasets
from random import randint
from datetime import datetime
import pandas as pd
from transformers import BertTokenizerFast,BertTokenizer, BertModel,BertConfig, BertForSequenceClassification, BertForMultipleChoice, Trainer, TrainingArguments
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer,RobertaTokenizerFast, RobertaForSequenceClassification
import argparse
import tensorflow as tf
# !pip install datasets
import datasets
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
import matplotlib.pyplot as plt
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,multilabel_confusion_matrix,plot_confusion_matrix
import warnings
warnings.filterwarnings("ignore")

iper_parameters={}
iper_parameters['MAX_LEN'] = 512 #200
iper_parameters['TRAIN_SIZE'] = 0.7
iper_parameters['TRAIN_BATCH_SIZE'] = 8
iper_parameters['VALID_BATCH_SIZE'] = 4
iper_parameters['EPOCHS'] = 1
iper_parameters['LEARNING_RATE'] = 1e-05

CHECKPOINT = "bert-base-multilingual-cased"

PROJECT_ID='********'

def get_args():
    '''Parses args.'''
    
    
    parser = argparse.ArgumentParser()
    parser.add_argument(
      '--epochs',
      required=False,
      default=iper_parameters['EPOCHS'],
      type=int,
      help='number of epochs')
    parser.add_argument(
      '--job_dir',
      required=True,
      type=str,
      help='bucket to store saved model, include gs://')
    args = parser.parse_args()
    return args

# Instantiates a client
def get_gs_content_file(f):
    client = storage.Client()
    bucket_name = PROJECT_ID+'-ml-vt-use-cases-topic-classification'
    file='Dataset/'+f
    bucket = client.get_bucket(bucket_name)
    blob = storage.blob.Blob(file,bucket)
    content = blob.download_as_string()
    return BytesIO(content)

def crea_label_vector(Data_Set):
    y_label_code_reduced=[]
    for index, row in Data_Set.iterrows():
        row_t=(list(row['target']))
        row_t=[int(x) for x in row_t]
        y_label_code_reduced.append(row_t)
    return y_label_code_reduced

def stat_ds(Data_Set,y_label_code_reduced):
    x=np.zeros(len(y_label_code_reduced[0]),dtype=int)
    for v in y_label_code_reduced:
        x=x+v
    c=0
    for v in x:
        if v==0:
            c+=1
    #percentuali sull'intero dataset
    xx=x.copy()
    z=[]
    for l in xx:
        z.append(str(round((l/len(Data_Set))*100,1))+'%')
    return x#,[float(x[:4]) for x in z]
   
def indx_permutati(Data_Set):
    random.seed(42)
    ind_ds=random.sample(range(len(Data_Set)), len(Data_Set))
    return ind_ds

def _red_list_bal(Data_Set,y_label_code_reduced):
    c=stat_ds(Data_Set,y_label_code_reduced)
    min_l=min(c)
    red=[0,0,0,0]
    ind=0
    for i in c:
        if i!=min_l:
            val=c[ind]#int((i/100)*len(Data_Set))
            red[ind]=val-min_l#int((min_l/100)*len(Data_Set))
        else:
            red[ind]=1
        ind+=1
    return red

def bilancia(Data_Set,y_label_code_reduced):
    size_prima=len(Data_Set)
    conta1=0
    conta2=0
    conta3=0
    conta4=0
    index=0
    red=_red_list_bal(Data_Set,y_label_code_reduced)
    indx_perm=indx_permutati(Data_Set)
    index_list=[]
    row_list=Data_Set['target'].tolist()
    for row_ind in indx_perm:
        row=row_list[row_ind]
        row_t=[int(x) for x in row]
        if sum(row_t)==1:
            if row_t[0]==1 and conta1<red[0]:
                index_list.append(row_ind)
                conta1+=1
            if row_t[1]==1 and conta1<red[1]:
                index_list.append(row_ind)
                conta2+=1
            if row_t[2]==1 and conta3<red[2]:
                index_list.append(row_ind)
                conta3+=1
            if row_t[3]==1 and conta4<red[3]:
                index_list.append(row_ind)
                conta4+=1
        index+=1
    Data_Set.drop(index_list, inplace=True)
    Data_Set.reset_index(drop=True)


def create_datasets_split():
    '''Creates a tf.data.Dataset for train and evaluation.'''
   
    processed_text_final = pandas.read_csv(get_gs_content_file('outputfile_text_processed.csv'))
    y_label_code = np.loadtxt(get_gs_content_file('numeric_label_topic.txt'), dtype=int)
    y_label_code=y_label_code.tolist()
    new_dict = np.load(get_gs_content_file("myDictionary_labels.npy"), allow_pickle='TRUE')
    new_dict.item()
    
    y_label_code_column=[]
    for row in y_label_code:
        lab=(''.join(map(str, row)))
        y_label_code_column.append(lab)
    Data_Set=pandas.DataFrame({'text':list(processed_text_final['text']),'target':y_label_code_column})
    
    delete=0
    start_row=35
    end_row=150 #len(Data_Set)
    delete=len(Data_Set[(Data_Set['text'].map(lambda x: len(x.split(' ')))> end_row) ])+len(Data_Set[(Data_Set['text'].map(lambda x: len(x.split(' ')))< start_row)])
    Data_Set = Data_Set[(Data_Set['text'].map(lambda x: len(x.split(' ')))<= end_row) ]
    Data_Set = Data_Set[(Data_Set['text'].map(lambda x: len(x.split(' ')))>= start_row) ]
    Data_Set = Data_Set.reset_index(drop=True)
    
    class_names_list=new_dict.item()
    target_names=[x for x in class_names_list.keys()]
    y_label_code_reduced=crea_label_vector(Data_Set)
    
    bilancia(Data_Set,y_label_code_reduced)
    
    y_label_code_reduced=crea_label_vector(Data_Set)
    
    X = list(Data_Set['text'])
    y = y_label_code_reduced
    new_df = pandas.DataFrame({'text':X, 'topic':y})
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

    train_dataset=new_df.sample(frac=iper_parameters['TRAIN_SIZE'],random_state=42).reset_index(drop=True)
    val_dataset=new_df.drop(train_dataset.index).reset_index(drop=True)

    validation_dataset=val_dataset.sample(frac=0.66,random_state=42).reset_index(drop=True)
    test_dataset=val_dataset.drop(validation_dataset.index).reset_index(drop=True)
    
    
    dd = datasets.DatasetDict({'train':datasets.Dataset(pa.Table.from_pandas(train_dataset)),'validation':datasets.Dataset(pa.Table.from_pandas(validation_dataset)) , 'test':datasets.Dataset(pa.Table.from_pandas(test_dataset))})
    
    return dd

def tokenize_function(examples, tokenizer):
    '''Tokenizes text examples.'''

    return tokenizer(examples['text'], padding='max_length', truncation=True)

def create_datasets():
    '''Creates a tf.data.Dataset for train and evaluation.'''
    
    raw_datasets = create_datasets_split()
    tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
    tokenized_datasets = raw_datasets.map((lambda examples: tokenize_function(examples, tokenizer)), batched=True)
    
    small_train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(1000))
    small_eval_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(1000))
    full_train_dataset = tokenized_datasets['train']
    full_eval_dataset = tokenized_datasets['test']
    
    tf_train_dataset = small_train_dataset.remove_columns(['text']).with_format("tensorflow")
    tf_eval_dataset = small_eval_dataset.remove_columns(['text']).with_format("tensorflow")
    
    train_features = {x: tf_train_dataset[x] for x in tokenizer.model_input_names}
    train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["topic"]))
    train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)
    
    eval_features = {x: tf_eval_dataset[x] for x in tokenizer.model_input_names}
    eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["topic"]))
    eval_tf_dataset = eval_tf_dataset.batch(8)
    
    return train_tf_dataset, eval_tf_dataset


def main():
    args = get_args()
    train_tf_dataset, eval_tf_dataset = create_datasets()
    
    model = TFAutoModelForSequenceClassification.from_pretrained(CHECKPOINT,num_labels=4)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=tf.metrics.BinaryAccuracy(),
    )
    
    model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=args.epochs)
    # model.fit(x=train_tf_dataset, validation_data=eval_tf_dataset, epochs=1)
    model.save(f'{args.job_dir}/model_output')
    model.save('model_output/')

if __name__ == "__main__":
    main()

Writing autopkg-codelab/trainer/task.py


---
## 2. Write a Requirements.txt file to specify additional dependencies of the ML code
---
These are additional dependencies for the model code not included in the predefined Vertex TensorFlow images such as TF-Hub, TensorFlow AdamW optimization and TensorFlow Text needed to import and work with pre-trained TensorFlow BERT models.

In [4]:
%%writefile {MODEL_DIR}/requirements.txt
tensorflow_text
tf-models-official
transformers
datasets==1.18.2
torch==1.10.1
torchvision==0.11.2
torchaudio==0.10.1

Writing autopkg-codelab/requirements.txt


---
# RUN
---

In [5]:
%%bash
PROJECT_ID='friendly-tower-338419'
BUCKET_NAME="gs://${PROJECT_ID}-ml-vt-use-cases-topic-classification"
echo $BUCKET_NAME

gs://friendly-tower-338419-ml-vt-use-cases-topic-classification


---
## 1. RUN ON CPU
---

In [ ]:
# %%bash
# cd autopkg-codelab
# gcloud services enable containerregistry.googleapis.com
# PROJECT_ID='friendly-tower-338419'
# BUCKET_NAME="gs://${PROJECT_ID}-models-vertexai"
# gsutil mb -l us-central1 $BUCKET_NAME

# BASE_CPU_IMAGE=us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest
# OUTPUT_IMAGE=$PROJECT_ID-local-package-cpu:latest

# gcloud ai custom-jobs local-run \
# --executor-image-uri=$BASE_CPU_IMAGE \
# --python-module=trainer.task \
# --output-image-uri=$OUTPUT_IMAGE \
# -- \
# --job_dir=$BUCKET_NAME \
# --epochs=1

---
## 2. RUN ON GPU
---

In [62]:
%%bash

cd mattia_gatto/text-mining-use-cases/src/use_cases/use_case_topic_classification
gcloud services enable containerregistry.googleapis.com
PROJECT_ID='*******'
BUCKET_NAME="gs://${PROJECT_ID}-vertexai-bert"
gsutil mb -l us-central1 $BUCKET_NAME

BASE_GPU_IMAGE=us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-7:latest

gcloud ai custom-jobs create \
--region=us-central1 \
--display-name=autopkg-codelab \
--args=--job_dir=$BUCKET_NAME \
--worker-pool-spec=machine-type=n1-standard-4,replica-count=1,accelerator-type=NVIDIA_TESLA_V100,executor-image-uri=$BASE_GPU_IMAGE,local-package-path=autopkg-codelab,python-module=trainer.task

---
# Automl endpoint request
---

In [86]:
from google.cloud import aiplatform
from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_text_classification_single_label_sample(
    project: str,
    endpoint_id: str,
    content: str,
    location: str = "us-central1",
    api_endpoint: str = "***",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    instance = predict.instance.TextClassificationPredictionInstance(
        content=content,
    ).to_value()
    instances = [instance]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)

    predictions = response.predictions
    for prediction in predictions:
        # print(" prediction:", dict(prediction))
        r=dict(prediction)
    return r

# [END aiplatform_predict_text_classification_single_label_sample]
def evaluete_automl(cont="YOUR_TEXT_CONTENT"):
    dict_pred=predict_text_classification_single_label_sample(
        project="****",
        endpoint_id="**********",
        location="us-central1",
        content=cont
    )
    dict_pred['confidences']
    dict_pred['displayNames']
    ris={dict_pred['confidences'][x]:dict_pred['displayNames'][x] for x in range (len(dict_pred['confidences']))}

    x=[y for y in list(ris.keys())]
    max_=0
    for i in x:
        if max_<=i:
            max_=i
    print('-'*50)
    print("'"+cont+"'",'--->',"<< "+ris[max_]+" >>")
    print('-'*50)
    return ris[max_]

In [87]:
cont="i like chicago"
label=evaluete_automl(cont=cont)

response
 deployed_model_id: 595205226533748736
--------------------------------------------------
'i like chicago' ---> << GOVERNMENT/SOCIAL >>
--------------------------------------------------
